In [1]:
import matplotlib.pyplot as plt
import MENDER
import numpy as np
import pandas as pd
import scanpy as sc
from sklearn.metrics import *

import pysodb

In [2]:
sodb = pysodb.SODB()
adata_dict = sodb.load_dataset('Allen2022Molecular_aging')

download experiment[MsBrainAgingSpatialDonor_10_0] in dataset[Allen2022Molecular_aging]


  3%|▎         | 1.50M/53.3M [00:06<03:48, 237kB/s] 


KeyboardInterrupt: 

In [3]:
# prepare input data

# specify slice_id and ground truth
adata_list = []
for si in adata_dict.keys():
    adata = adata_dict[si]
    adata.obs['slice_id'] = si
    adata_list.append(adata)
adata_raw = adata_list[0].concatenate(adata_list[1:])
adata_raw.obs['slice_id'] = adata_raw.obs['slice_id'].astype('category')
adata_raw.obs['gt'] = adata_raw.obs['tissue'].astype('category')
adata_raw.obs['ct'] = adata_raw.obs['clust_annot'].astype('category')

/opt/anaconda3/envs/mender/lib/python3.9/site-packages/anndata/_core/anndata.py:1763: FutureWarning: The AnnData.concatenate method is deprecated in favour of the anndata.concat function. Please use anndata.concat instead.

See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  warnings.warn(
/opt/anaconda3/envs/mender/lib/python3.9/site-packages/anndata/_core/merge.py:1015: FutureWarning: The default value of 'ignore' for the `na_action` parameter in pandas.Categorical.map is deprecated and will be changed to 'None' in a future version. Please set na_action to the desired value to avoid seeing this warning
  concat_indices = concat_indices.str.cat(label_col.map(str), sep=index_unique)
/opt/anaconda3/envs/mender/lib/python3.9/site-packages/anndata/_core/merge.py:1015: FutureWarning: The default value of 'ignore' for the `na_action` parameter in pandas.Categorical.map is deprecated and will be changed to 'None' in a future version. Please set na_ac

In [4]:
batch_obs = 'slice_id'
gt_obs = 'gt'

In [ ]:
# run MENDER

import time
time_st = time.time()
# input parameters of MENDER
scale = 6
radius = 15

# estimate number of domains
n_cls = np.unique(adata_raw.obs[gt_obs]).shape[0]




adata = adata_raw.copy()


# main body of MENDER
msm = MENDER.MENDER(
    adata,
    batch_obs = batch_obs,
    # determine which cell state to use
    # we use the cell state got by Leiden
    ct_obs='ct',
    random_seed=0
)


# set the MENDER parameters


msm.prepare()
msm.set_MENDER_para(
    # default of n_scales is 6
    n_scales=scale,

    # for single cell data, nn_mode is set to 'radius'
    nn_mode='radius',

    # default of n_scales is 15 um (see the manuscript for the analysis).
    # MENDER also provide a function 'estimate_radius' for estimating the radius
    nn_para=radius,

)
# construct the context representation
msm.run_representation_mp(
    200
    # the number of processings
)

# set the spatial clustering parameter
# positive values for the expected number of domains
# negative values for the clustering resolution
# msm.run_clustering_normal(n_cls)

# using this res, the output is 8 domains
msm.run_clustering_normal(-0.)


time_cost = time.time()-time_st

default number of process is 200
total batch: 31, running batch MsBrainAgingSpatialDonor_10_0
scale 0, median #cells per radius (r=15): 2.0
total batch: 31, running batch MsBrainAgingSpatialDonor_10_1
scale 1, median #cells per radius (r=15): 3.0
scale 0, median #cells per radius (r=15): 1.0
scale 2, median #cells per radius (r=15): 5.0
scale 1, median #cells per radius (r=15): 3.0
scale 3, median #cells per radius (r=15): 7.0
total batch: 31, running batch MsBrainAgingSpatialDonor_10_2
scale 2, median #cells per radius (r=15): 4.0
scale 0, median #cells per radius (r=15): 2.0
scale 4, median #cells per radius (r=15): 8.0
scale 3, median #cells per radius (r=15): 6.0
scale 1, median #cells per radius (r=15): 3.0
scale 5, median #cells per radius (r=15): 10.0
scale 4, median #cells per radius (r=15): 8.0
scale 2, median #cells per radius (r=15): 5.0
total batch: 31, running batch MsBrainAgingSpatialDonor_11_0
scale 3, median #cells per radius (r=15): 7.0
scale 5, median #cells per radiu

In [3]:
msm.run_clustering_normal(-0.5)

NameError: name 'msm' is not defined

In [4]:
print(f'cost {time_cost} s for {adata_raw.shape[0]} cells')


NameError: name 'time_cost' is not defined

In [5]:
# the plot function has two parameters
# obs: the observation to plot
# gt_obs: the ground truth observation to compute NMI and ARI, can be set to None if not available
msm.output_cluster_all(obs='MENDER',obs_gt=gt_obs)
print('MENDER prediction')

NameError: name 'msm' is not defined

In [ ]:
# the plot function has two parameters
# obs: the observation to plot
# gt_obs: the ground truth observation to compute NMI and ARI, can be set to None if not available

msm.output_cluster_all(obs=gt_obs,obs_gt=None)
print('ground truth')